## Triggers pulse for X-ray setup

In [1]:
import serial
import serial.tools.list_ports
import time

# CONFIGURATION
PORT        = "COM1"     # ← change this to your port
BAUDRATE    = 9600
KV_SET      = 70         # fixed kV
MAX_UA      = 250        # your SourceBlock's max µA (e.g. 250 µA)
START_UA    = 20         # starting current (µA)
STEP_UA     = 20         # current step (µA)
END_UA      = 200        # final current (µA)
ON_TIME     = 10         # seconds X-ray ON
OFF_TIME    = 10         # seconds X-ray OFF
SETTLE_TIME = 2          # seconds to wait after X-Ray ON before reading
WAIT_TIME   = 20         # countdown time to start pulse measurement

def detect_serial_ports():
    ports = serial.tools.list_ports.comports()
    if not ports:
        print("No serial ports found.")
    else:
        print("Available serial ports:")
        for p in ports:
            print(f"  {p.device}: {p.description}")

def to_counts(value, full_scale, counts=4095):
    """Map a physical value onto 0–counts."""
    return int(round(value / full_scale * counts))

def send(ser, cmd):
    ser.write(cmd.encode())
    time.sleep(0.05)

def read_counts(ser, cmd):
    """
    Send cmd<CR> and read back up to the next <CR>,
    parse as integer counts (0–4095).
    """
    ser.reset_input_buffer()
    send(ser, cmd + "\r")
    raw = ser.read_until(b'\r')
    text = raw.decode('ascii', errors='ignore').strip()
    try:
        return int(text)
    except ValueError:
        print("⚠️ unexpected reply from", cmd, "→", repr(raw))
        return None

def initialize(ser):
    """Clear faults per §2.3 of the DI-RS232A spec."""
    send(ser, "CPA1111100\r")
    send(ser, "RESPA0\r")
    send(ser, "SETPA1\r")
    time.sleep(0.1)
    send(ser, "RESPA1\r")

def set_kv(ser, kv):
    cnt = to_counts(kv, full_scale=80.0)
    send(ser, f"VA{cnt:04d}\r")
    print(f"> SET KV → {kv} kV (VA{cnt:04d})")

def get_kv(ser):
    cnt = read_counts(ser, "RD0")
    if cnt is None:
        return None
    kv = cnt / 4095 * 80.0
    print(f"< Voltage status: KV → {kv:.2f} kV (raw {cnt})")
    return kv

def set_ua(ser, ua):
    cnt = to_counts(ua, full_scale=MAX_UA)
    send(ser, f"VB{cnt:04d}\r")
    print(f"> SET µA → {ua} µA (VB{cnt:04d})")

def get_ua(ser):
    cnt = read_counts(ser, "RD1")
    if cnt is None:
        return None
    ua = cnt / 4095 * MAX_UA
    print(f"< Current status: µA → {ua:.2f} µA (raw {cnt})")
    return ua

def xray_on(ser):
    send(ser, "SETPA0\r")
    print("→ X-Ray ON")

def xray_off(ser):
    send(ser, "RESPA0\r")
    print("→ X-Ray OFF")

def pulse_sequence(ser):
    initialize(ser)
    xray_off(ser)                  # start with tube OFF
    time.sleep(OFF_TIME)

    print("X-ray pulse measurement is about to start:")
    for i in range(WAIT_TIME, 0, -1):
        print(f"Starting in {i} seconds...", end="\r")
        time.sleep(1)
    print("Starting now! \n")

    # 1) Program kV once
    set_kv(ser, KV_SET)

    # 2) Loop through current steps
    for ua in range(START_UA, END_UA + 1, STEP_UA):
        set_ua(ser, ua)

        xray_on(ser)
        # let HV & tube current ramp
        time.sleep(SETTLE_TIME)
        # now read the actual kV & µA
        get_kv(ser)
        get_ua(ser)

        # finish out the ON interval
        time.sleep(ON_TIME - SETTLE_TIME)

        xray_off(ser)
        time.sleep(OFF_TIME)
        print()

if __name__ == "__main__":
    detect_serial_ports()
    try:
        with serial.Serial(PORT, BAUDRATE, timeout=1) as ser:
            print(f"Opened {PORT} @ {BAUDRATE} baud\n")
            try:
                pulse_sequence(ser)
            except KeyboardInterrupt:
                print("\n⚠️ Interrupted by user.")
            finally:
                # guaranteed safety shut-off
                xray_off(ser)
                print("X-Ray turned OFF for safety.")
    except Exception as e:
        print("ERROR:", e)


Available serial ports:
  COM1: Communications Port (COM1)
  COM2: Communications Port (COM2)
Opened COM1 @ 9600 baud

→ X-Ray OFF
X-ray pulse measurement is about to start:
Starting now! seconds....

> SET KV → 70 kV (VA3583)
> SET µA → 20 µA (VB0328)
→ X-Ray ON
< Voltage status: KV → 69.88 kV (raw 3577)
< Current status: µA → 20.02 µA (raw 328)

⚠️ Interrupted by user.
→ X-Ray OFF
X-Ray turned OFF for safety.
